In [1]:
from functions import *

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
plt.style.use('fivethirtyeight')
#sns.set()
#pd.options.display.max_columns = 100

# Read Master data

In [2]:
master_df = pd.read_csv('master_final_cleaned.csv')
master_df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,client_credit_SK_DPD_DEF_min_count,client_credit_SK_DPD_DEF_min_mean,client_credit_SK_DPD_DEF_min_max,client_credit_SK_DPD_DEF_min_min,client_credit_SK_DPD_DEF_min_sum,client_credit_SK_DPD_DEF_sum_count,client_credit_SK_DPD_DEF_sum_mean,client_credit_SK_DPD_DEF_sum_max,client_credit_SK_DPD_DEF_sum_min,client_credit_SK_DPD_DEF_sum_sum
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,9461,-637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,16765,-1188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,19046,-225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,19005,-3039,...,6.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,19932,-3038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# instantiate xgboost model

In [3]:
#instantiate XGB Regressor
xgbr = xgb.XGBRegressor(n_jobs=-1,objective='binary:logistic', eval_metric='auc')

#Hyper Parameter Optimization
params = {
   "learning_rate": [0.05, 0.10, 0.20],
    "max_depth": [4, 8, 12],
    "min_child_weight": [1],
    "gamma": [0.0],
    "colsample_bytree": [0.3],
    "n_estimators" : [100]
}

random_search = RandomizedSearchCV(xgbr,
                                   param_distributions=params,
                                   scoring='roc_auc',
                                   n_jobs=-1,cv=1,verbose=3)

# Build Xgboost Model

In [4]:
# Prepare data to run rhe model
X = master_df.drop(['TARGET','SK_ID_CURR'],axis=1).copy()
y = master_df['TARGET'].copy()

# free memory
gc.enable()
del master_df
gc.collect()

#split data into train,test (80,20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=7)

# Standardization
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

random_search.fit(X_train_scaled,y_train)
y_pred_test = random_search.predict(X_test_scaled)
y_pred_train = random_search.predict(X_train_scaled)

best_model = random_search.best_estimator_


print("train score:", r2_score(y_train,y_pred_train))
print("test score:", r2_score(y_test,y_pred_test))
important_features = print_importance_features(best_model, X_train.columns)

0

MemoryError: Unable to allocate 330. MiB for an array with shape (215257, 1607) and data type bool